# Prompt Engineering com IMDB

Juvenal Jr. - 242160

> Utilizar o groq.com para usar a API do Llama 3.1-8b para fazer análise de sentimentos do IMDB.

In [ ]:
!pip install datasets
!pip install groq
!pip install tqdm boto3 requests regex sentencepiece sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━

In [ ]:
import os # Operações com o SO (ler variáveis de ambiente)
import random # Operações randômicas
from concurrent.futures import ThreadPoolExecutor # Paralelização
import threading # Paralelização
import time # Temporização
from typing import Optional, List # Type hints
import datasets # Obter o dataset IMDB
import groq # API para utilizar o Llama 3
import tqdm # Print de progresso
import torch # ML
import pandas # Data manipulation

## Interface para o Groq

Para realizar a inferência utilizando a API do Groq, criamos uma classe:

In [ ]:
class GroqInterface:
    '''
    Interface para utilizar a API Groq.
    '''

    _client = None  # Armazena uma instância única do cliente da API Groq (padrão Singleton).

    LLAMA3_31_8B_INSTANT = "llama-3.1-8b-instant"  # Define o modelo correto a ser utilizado.

    rate_lock = threading.Lock()  # Cria um bloqueio para controlar o limite de taxa em cenários multithreading.

    def __init__(self, model: Optional[str] = None):
        '''
        Construtor da classe GroqInterface.
        '''
        if GroqInterface._client is None:
            api_key = os.environ.get("GROQ_API_KEY")
            if api_key is None:
                raise RuntimeError("A chave de API não está nas variáveis de ambiente ('GROQ_API_KEY' não foi definida).")
            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_31_8B_INSTANT  # Use o modelo `llama-3.1-8b-instant`.
        self._model = model

    def __call__(self, prompt: str) -> str:
        '''
        Gera uma resposta do modelo.
        '''
        done = False
        while not done:
            try:
                with GroqInterface.rate_lock:
                    chat_completion = GroqInterface._client.chat.completions.create(
                        messages=[
                            {"role": "user", "content": prompt}
                        ],
                        model=self._model,
                    )
                done = True
            except groq.RateLimitError:
                time.sleep(2)
            except groq.NotFoundError as exception:
                raise exception
            except Exception as exception:
                raise exception

        return chat_completion.choices[0].message.content


In [ ]:
# Definindo a variável de ambiente com sua chave de API
os.environ['GROQ_API_KEY'] = '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'

# Agora você pode inicializar o GroqInterface sem erros
groq_interface = GroqInterface()


In [ ]:
models = GroqInterface._client.models.list()
for model in models.data:
    print(model.id)


distil-whisper-large-v3-en
llama-3.1-8b-instant
llama3-8b-8192
llama-3.2-1b-preview
llama3-groq-70b-8192-tool-use-preview
mixtral-8x7b-32768
llama-3.2-11b-text-preview
llama3-groq-8b-8192-tool-use-preview
gemma-7b-it
llava-v1.5-7b-4096-preview
llama-3.2-3b-preview
whisper-large-v3
llama-3.2-90b-text-preview
llama3-70b-8192
llama-3.1-70b-versatile
llama-guard-3-8b
gemma2-9b-it


In [ ]:
groq_interface("Hello!")

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [ ]:
POSITIVE = 1
NEGATIVE = 0

In [ ]:
class GroqSentimentInterface(GroqInterface):
    '''
    Classe que estende a GroqInterface, adicionando um pós-processamento
    para análise de sentimentos.
    '''

    def __call__(self, prompt: str) -> int:
        '''
        Gera a resposta do modelo para análise de sentimentos.

        Se o modelo fornecer uma resposta ambígua (contendo tanto termos positivos
        quanto negativos), um valor aleatório é gerado.

        Args:
            prompt (str): o prompt enviado para o modelo.

        Retorna:
            int: resposta do modelo. Retorna POSITIVE se o sentimento for positivo,
                 ou NEGATIVE em caso contrário.
        '''

        # Chama o método __call__ da classe base (GroqInterface) para obter a resposta do modelo.
        response = super().__call__(prompt)

        # Converte a resposta para letras minúsculas para facilitar a comparação.
        response = response.lower()

        # Verifica se a resposta contém a palavra "positive" e não contém "negative".
        if "positive" in response and "negative" not in response:
            return POSITIVE  # Retorna POSITIVE se a condição for verdadeira.

        # Verifica se a resposta contém a palavra "negative" e não contém "positive".
        if "negative" in response and "positive" not in response:
            return NEGATIVE  # Retorna NEGATIVE se a condição for verdadeira.

        # Se a resposta for ambígua (contém tanto positivo quanto negativo),
        # escolhe aleatoriamente entre POSITIVE e NEGATIVE.
        return random.choice([POSITIVE, NEGATIVE])


In [ ]:
groq_sentiment = GroqSentimentInterface()

## IMDB Prompt Engineering

In [ ]:
executor = ThreadPoolExecutor(max_workers=2)
# Cria um executor com pool de threads, permitindo execução paralela de até 2 tarefas.

trainbase_future = executor.submit(datasets.load_dataset, "imdb", split="train")
# Envia uma tarefa para o executor carregar o dataset de treino do IMDB de forma assíncrona,
# ou seja, em uma thread separada. O resultado dessa tarefa será obtido posteriormente.

test_future = executor.submit(datasets.load_dataset, "imdb", split='test')
# Envia outra tarefa para o executor carregar o dataset de teste do IMDB de forma assíncrona.

trainbase_dataset = trainbase_future.result()
# Obtém o resultado da tarefa assíncrona de carregar o dataset de treino (espera até que a tarefa seja concluída).

testbase_dataset = test_future.result()
# Obtém o resultado da tarefa assíncrona de carregar o dataset de teste (espera até que a tarefa seja concluída).

train_val_dataset = trainbase_dataset.train_test_split(test_size=100, shuffle=True, seed=78)
# Divide o dataset de treino em duas partes: treino e validação. Seleciona 300 amostras aleatórias para validação,
# embaralhando os dados antes de fazer a divisão. A seed garante a reprodução do embaralhamento.

discard_test_dataset = testbase_dataset.train_test_split(test_size=100, shuffle=True, seed=78)
# Divide o dataset de teste, descartando parte dos dados para obter um conjunto menor de teste
# com 300 amostras. O embaralhamento também é controlado por uma seed.

train_dataset = train_val_dataset["train"]
# Extrai o conjunto de treino da divisão feita anteriormente.

val_dataset = train_val_dataset["test"]
# Extrai o conjunto de validação da divisão feita anteriormente.

test_dataset = discard_test_dataset["test"]
# Extrai o novo conjunto de teste, a partir da divisão anterior.


In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(24900, 100, 100)

## Zero-shot

Para a técnica de zero-shot, precisamos apenas preparar um prompt que solicita a classificação ao modelo:

In [ ]:
base_prompt_zero = '''Classify if the movie review is POSITIVE or NEGATIVE:
                Review:
                {review}

                Sentiment:
                POSITIVE OR NEGATIVE:
                '''
# Este é um template de prompt utilizado para solicitar ao modelo que classifique uma resenha de filme.
# A variável {review} será substituída pelo texto da resenha do filme.
# O modelo deve indicar se o sentimento da resenha é POSITIVO ou NEGATIVO.

In [ ]:
prompt = base_prompt_zero.format(review=train_dataset[-1]["text"])
# Substitui o placeholder {review} no template base_prompt_zero pelo texto da última resenha
# presente no dataset de treino (train_dataset[-1]["text"]).

groq_sentiment(prompt), train_dataset[-1]["label"]
# Envia o prompt gerado para o modelo groq_sentiment, que faz a análise de sentimento
# (retorna POSITIVE ou NEGATIVE).
# Ao mesmo tempo, exibe o rótulo real (label) da última resenha no dataset de treino,
# para comparar o resultado gerado pelo modelo com o rótulo original.


(1, 1)

Preparamos a função para realizar a avaliação de um sample:

In [ ]:
def evaluate_zero(text:str, label:int) -> bool:
    '''
    Avalia a resposta do modelo em um cenário de zero-shot (sem treino específico para o conjunto de dados).

    Args:
        text (str): resenha (texto) a ser avaliada.
        label (int): rótulo esperado da resenha (sentimento correto, 0 para negativo e 1 para positivo).

    Retorna:
        bool: True se o modelo classificar corretamente, False caso contrário.
    '''
    # Formata o prompt usando o template base, inserindo a resenha no lugar do {review}.
    prompt = base_prompt_zero.format(review=text)

    # Obtém o resultado da análise de sentimento usando o modelo groq_sentiment.
    result = groq_sentiment(prompt)

    # Compara o resultado gerado pelo modelo com o rótulo esperado e retorna True se estiver correto.
    return result == label

E calculamos a acurácia utilizando os dados de validação:

In [ ]:
executor = ThreadPoolExecutor(max_workers=4) # Mais trabalhadores -> Mais exceções de RateLimit
# Cria um executor com um pool de threads que pode executar até 4 tarefas em paralelo.
# Se o número de trabalhadores (threads) for muito alto, pode aumentar a ocorrência de exceções de RateLimit,
# devido ao número elevado de requisições simultâneas à API.

futures = []
# Lista para armazenar as tarefas futuras (operações assíncronas).

for data in val_dataset:
    future = executor.submit(evaluate_zero, **data)
    # Envia a função evaluate_zero para ser executada em paralelo usando o executor.
    # Cada item do val_dataset contém os dados que são passados como argumentos (text e label) para a função evaluate_zero.
    futures.append(future)
    # Adiciona cada tarefa assíncrona (futura) à lista de futures.

correct_zero = 0
# Inicializa o contador de classificações corretas.

for future in tqdm.tqdm(futures):
    correct_zero += future.result()
    # Para cada tarefa futura, obtém o resultado da função evaluate_zero (True ou False).
    # Se o resultado for True (classificação correta), incrementa o contador correct_zero.
    # A barra de progresso tqdm é utilizada para mostrar o progresso da execução das tarefas.


100%|██████████| 100/100 [05:30<00:00,  3.31s/it]


In [ ]:
accuracy_zero = correct_zero/len(val_dataset)
print(f"Acurácia - Zero-shot - Validação: {accuracy_zero*100}%")

Acurácia - Zero-shot - Validação: 90.0%


## Few-shot

In [ ]:
raw_prompt_few = '''Classify if the movie review is positive or negative:
                # Inicia o prompt, pedindo ao modelo para classificar se a resenha do filme é positiva ou negativa.

                Review:
                Movie review
                # Aqui o prompt estabelece o campo "Review" e coloca um texto genérico "Movie review", que será substituído mais tarde por exemplos reais de resenhas.

                Sentiment:
                ONLY POSITIVE OR NEGATIVE
                # Especifica que a resposta do modelo deve ser limitada a "POSITIVE" ou "NEGATIVE", restringindo as possibilidades de resposta.

                Classify if this movie review is positive or negative:
                Review:
                {example1}
                # Aqui é fornecido o primeiro exemplo de resenha de filme que será injetado via o placeholder "{example1}".

                Sentiment:
                {response1}
                # O sentimento correspondente ao primeiro exemplo é inserido no placeholder "{response1}", indicando se é "POSITIVE" ou "NEGATIVE".

                Classify if this movie review is positive or negative:
                Review:
                {example2}
                # Fornece o segundo exemplo de resenha de filme, também usando um placeholder "{example2}".

                Sentiment:
                {response2}
                # O sentimento do segundo exemplo é inserido aqui através de "{response2}", que será preenchido com "POSITIVE" ou "NEGATIVE".

                Classify if this movie review is positive or negative:
                Review:
                {{review}}
                # Esta parte final do prompt solicita a classificação da nova resenha, que será substituída pelo texto real através do placeholder "{{review}}".

                Sentiment:
                # Aqui, o modelo deve prever se o sentimento da resenha é "POSITIVE" ou "NEGATIVE".
                '''


In [ ]:
positive_example = None
negative_example = None
# Inicializa as variáveis 'positive_example' e 'negative_example' como None.
# Elas serão usadas para armazenar exemplos de resenhas classificadas como positivas e negativas, respectivamente.

i = 0
# Inicializa o contador 'i' com valor 0. Este será usado para percorrer os dados do conjunto de treino.

while positive_example is None or negative_example is None:
    # O loop 'while' continua até que ambas as variáveis 'positive_example' e 'negative_example'
    # sejam preenchidas com exemplos, ou seja, enquanto pelo menos uma delas for 'None'.

    if train_dataset[i]["label"] == POSITIVE:
        positive_example = train_dataset[i]
        # Se o rótulo do exemplo atual do dataset for positivo (representado pela constante 'POSITIVE'),
        # armazena esse exemplo em 'positive_example'.
    else:
        negative_example = train_dataset[i]
        # Caso contrário, se o rótulo for negativo, armazena o exemplo em 'negative_example'.

    i += 1
    # Incrementa o valor de 'i' para avançar para o próximo exemplo no conjunto de treino na próxima iteração.


In [ ]:
base_prompt_few = raw_prompt_few.format(example1=positive_example["text"], response1="POSITIVE",
                                        example2=negative_example["text"], response2="NEGATIVE")
# O método 'format' é usado para preencher os placeholders na string 'raw_prompt_few'.
# - 'example1=positive_example["text"]' insere o texto do exemplo positivo armazenado em 'positive_example["text"]' no placeholder '{example1}'.
# - 'response1="POSITIVE"' insere a string "POSITIVE" no placeholder '{response1}', que indica o sentimento associado ao primeiro exemplo.
# - 'example2=negative_example["text"]' insere o texto do exemplo negativo armazenado em 'negative_example["text"]' no placeholder '{example2}'.
# - 'response2="NEGATIVE"' insere a string "NEGATIVE" no placeholder '{response2}', que indica o sentimento associado ao segundo exemplo.
# O resultado é que a string 'raw_prompt_few' se torna um prompt preenchido com exemplos reais de resenhas positivas e negativas.

print(base_prompt_few)
# Imprime o 'base_prompt_few', que agora contém o prompt formatado com as resenhas e suas classificações.


Classify if the movie review is positive or negative:
                # Inicia o prompt, pedindo ao modelo para classificar se a resenha do filme é positiva ou negativa.

                Review:
                Movie review
                # Aqui o prompt estabelece o campo "Review" e coloca um texto genérico "Movie review", que será substituído mais tarde por exemplos reais de resenhas.

                Sentiment:
                ONLY POSITIVE OR NEGATIVE
                # Especifica que a resposta do modelo deve ser limitada a "POSITIVE" ou "NEGATIVE", restringindo as possibilidades de resposta.

                Classify if this movie review is positive or negative:
                Review:
                but I want to say I cannot agree more with Moira.<br /><br />What a wonderful film.<br /><br />I was thinking about it just this morning, wanting to give advice to some dopey sod who'd lost money on his debit card through fraud, and wanted to say 'Keep thy money in thine pocket' and

In [ ]:
prompt = base_prompt_few.format(review=train_dataset[-1]["text"])
# Usa o método 'format()' novamente para preencher o placeholder '{{review}}' dentro da string 'base_prompt_few' com o texto da última resenha do 'train_dataset'.
# 'train_dataset[-1]["text"]' seleciona a última resenha no conjunto de treinamento (o índice [-1] refere-se ao último item da lista).
# O resultado é que o 'prompt' agora contém a string formatada com dois exemplos anteriores (um positivo e um negativo) e a nova resenha do dataset.

groq_sentiment(prompt), train_dataset[-1]["label"]
# Chama a função 'groq_sentiment()', passando o 'prompt' como argumento. Provavelmente, essa função realiza a classificação de sentimento (ou análise de sentimento) com base no prompt fornecido.
# Em seguida, 'train_dataset[-1]["label"]' retorna o rótulo real (ou seja, o sentimento real) da última resenha no conjunto de treinamento.
# O resultado dessa linha retorna uma tupla contendo:
# 1. O sentimento previsto pela função 'groq_sentiment(prompt)'.
# 2. O rótulo real da resenha (positivo ou negativo) a partir do 'train_dataset[-1]["label"]'.


(1, 1)

In [ ]:
def evaluate_few(text: str, label: int) -> bool:
    '''
    Avalia a resposta do modelo usando few-shot learning.

    Args:
        text (str): A resenha que será avaliada.
        label (int): O rótulo esperado para a resenha (pode ser uma constante, como 0 para "NEGATIVE" e 1 para "POSITIVE").

    Returns:
        bool: Retorna True se o modelo classificar a resenha corretamente, caso contrário, retorna False.
    '''

    # Preenche o template base (base_prompt_few) com a resenha fornecida (text).
    # O placeholder '{{review}}' no 'base_prompt_few' é substituído pelo texto da resenha.
    prompt = base_prompt_few.format(review=text)

    # Chama a função 'groq_sentiment()', que recebe o 'prompt' formatado como entrada
    # e retorna o resultado da classificação de sentimento, seja "POSITIVE" ou "NEGATIVE".
    result = groq_sentiment(prompt)

    # Retorna True se a previsão do modelo ('result') for igual ao rótulo esperado ('label').
    # Caso contrário, retorna False, indicando que a classificação estava incorreta.
    return result == label


In [ ]:
evaluate_few(**train_dataset[-1])
# Desempacota o último item do 'train_dataset' como argumentos nomeados para 'evaluate_few'.
# Isso equivale a passar 'text=train_dataset[-1]["text"]' e 'label=train_dataset[-1]["label"]' para a função.
# Avalia se o modelo classifica corretamente a última resenha do dataset com base no rótulo esperado.


True

In [ ]:
executor = ThreadPoolExecutor(max_workers=4)
# Cria um pool de threads com um máximo de 4 threads (trabalhadores) simultâneas.
# Um número maior de trabalhadores pode resultar em mais exceções de RateLimit, indicando que as requisições estão excedendo o limite de taxa permitido.

futures = []
# Inicializa uma lista vazia para armazenar os objetos 'Future', que representam os resultados das tarefas executadas de forma assíncrona.

for data in val_dataset:
    future = executor.submit(evaluate_few, **data)
    # Para cada item no conjunto de validação (val_dataset), a função 'evaluate_few' é submetida ao 'ThreadPoolExecutor' para ser executada em paralelo.
    # Os dados são desempacotados (**data) para serem passados como argumentos nomeados para a função 'evaluate_few'.
    # A função 'submit' retorna um objeto 'Future', que será usado para recuperar o resultado da execução da função.
    futures.append(future)
    # Adiciona cada 'Future' à lista 'futures' para ser processado posteriormente.

correct_few = 0
# Inicializa um contador para armazenar o número de classificações corretas.

for future in tqdm.tqdm(futures):
    correct_few += future.result()
    # Para cada 'Future' na lista 'futures', usa o método 'result()' para bloquear e esperar o término da execução da função.
    # 'future.result()' retorna o valor booleano de 'evaluate_few' (True se a classificação foi correta, False se foi incorreta).
    # Se o resultado for True, incrementa 'correct_few', contando as classificações corretas.


100%|██████████| 100/100 [34:11<00:00, 20.51s/it]


In [ ]:
accuracy_few = correct_few / len(val_dataset)
# Calcula a acurácia da validação few-shot.
# 'correct_few' representa o número total de classificações corretas, e 'len(val_dataset)' é o número total de exemplos no conjunto de validação.
# A acurácia é obtida dividindo o número de classificações corretas pelo número total de exemplos.

print(f"Acurácia - Few-shot - Validação: {accuracy_few*100}%")
# Exibe a acurácia em formato percentual.
# Multiplica 'accuracy_few' por 100 para converter a proporção em uma porcentagem e imprime o valor formatado.


Acurácia - Few-shot - Validação: 87.0%


## Comparação e Teste


Técnica | Acurácia de Validação | Tempo
-|-|-
Zero-shot|90%|3 min 31 s
Few-shot|87%|34 min 11 s
